I-27-6 Feynamn Lectures on Physics

https://www.feynmanlectures.caltech.edu/I_27.html

Optical geometry - derivation of focal length based on distances (d1,d2) from lens and refraction index n 

foc = 1/(1/d1+n/d2) ==> foc/d1 + foc*n/d2 = 1 ==> foc*d2 + foc * n * d1 - d1 * d2=0


In [23]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using SymbolicUtils # for symbolic simplification

functions for generating monomials

In [24]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [5]:
@polyvar tvq n VQ top1 top2 h d1 d2 r foc   # 

x   =   [ tvq n VQ top1 top2 h d1 d2 r foc   ] #

1×10 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 tvq  n  VQ  top1  top2  h  d1  d2  r  foc

In [17]:
axioms= 
[
    tvq - (n-1) * VQ,
    top1 *  (2 * d1) - h^2 ,         # top1 - h^2 / (2 * d1),
    top2 * (2 * d2) - (n * h^2),     # top2 - (n * h^2) / (2 * d2),
    VQ * (2 * r) - h^2,              # VQ - h^2 / (2 * r),
    top1 + top2 - tvq,
    foc*(n-1) - r,                   # foc - r/(n-1)
    #d1>0,
    #d2>0,
    #h>0,
    #VQ>0,
    #r>0,
    #top1>0,
    #n>1,
    #tvq>0,
    #top2>0,
]
    

6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 VQ + tvq - nVQ
 -h² + 2top1d1
 2top2d2 - nh²
 -h² + 2VQr
 top2 + top1 - tvq
 -foc - r + nfoc

In [34]:
deg = 3
deg_overall = 14
theDegrees = [ 2   2  2   2    2   2  3  3 2  2 ] 
            # tvq  n  VQ top1 top2 h d1 d2 r foc  

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

#model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([d1 d2 n  foc], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [foc]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(93784,), (93784,), (93784,), (93784,), (93784,), (93784,)]


In [29]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#143 (generic function with 1 method)

In [30]:
value_q = value_poly(q)
value_q

foc² + rfoc - nfoc²

In [31]:
q_sim = simplify(value_q)

foc² + rfoc - nfoc²

which can be simplified to (foc^2*h^2*r*(d1*d2 + d2*r - d1*d2*n + d1*n*r))/2



In [27]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[d1d2r²foc², -0.5d2r²foc², -0.5d1r²foc², -0.5d1d2rfoc² + 0.5nd1d2rfoc², d1d2r²foc², 0.0]


6-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 d1d2r²foc²
 -0.5d2r²foc²
 -0.5d1r²foc²
 -0.5d1d2rfoc² + 0.5nd1d2rfoc²
 d1d2r²foc²
 0.0